In [22]:
import pandas as pd
import os


In [23]:
reddit_df = pd.read_csv("../chatgpt-reddit-comments.csv")

In [24]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT
3,3,iztfhtb,t3_zj2aeu,"Worked on me, ngl.",r/ChatGPT
4,4,izu2as9,t3_zj2aeu,"Certified 10/10, must-see moment. It really di...",r/ChatGPT
...,...,...,...,...,...
52411,16668,j5m0v6m,t3_10jmvpj,"Read the T.O.S., you'll thank me later",r/technology
52412,16669,j5m6aj0,t1_j5m0v6m,What am I missing here... https://openai.com/t...,r/technology
52413,16670,j5nylax,t1_j5m0v6m,What does ChatGTP think of its own TOS?,r/technology
52414,16671,j5mwpdr,t1_j5m6aj0,Don't know what they're referring to in the TO...,r/technology


In [25]:
academic_words_list = ["class", "lecture", "professor", "prof", "homework", "hw", "exam", "examinations", "assignment", 
"study", "degree", "gpa", "cap", "scholarship", "scholar", "research", "thesis", "lab", "campus", "graduation", "grad", "syllabus",
"textbook", "textbooks", "student", "students", "academic", "acad", "acads", "registrar", "tuition", "coursework", "course", "attendance",
"faculty", "teacher", "teach", "learn", "internship", "intern", "library", "peer", "peers", "school", "schools", "university", "uni", "college", "colleges"
]

In [26]:
reddit_df = reddit_df.drop_duplicates(subset=["comment_body"])

In [27]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT
3,3,iztfhtb,t3_zj2aeu,"Worked on me, ngl.",r/ChatGPT
4,4,izu2as9,t3_zj2aeu,"Certified 10/10, must-see moment. It really di...",r/ChatGPT
...,...,...,...,...,...
52411,16668,j5m0v6m,t3_10jmvpj,"Read the T.O.S., you'll thank me later",r/technology
52412,16669,j5m6aj0,t1_j5m0v6m,What am I missing here... https://openai.com/t...,r/technology
52413,16670,j5nylax,t1_j5m0v6m,What does ChatGTP think of its own TOS?,r/technology
52414,16671,j5mwpdr,t1_j5m6aj0,Don't know what they're referring to in the TO...,r/technology


In [28]:
reddit_df = reddit_df.dropna(subset=["comment_body"])

In [29]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT
3,3,iztfhtb,t3_zj2aeu,"Worked on me, ngl.",r/ChatGPT
4,4,izu2as9,t3_zj2aeu,"Certified 10/10, must-see moment. It really di...",r/ChatGPT
...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"Scary thing is, in this future text won’t be t...",r/technology
52411,16668,j5m0v6m,t3_10jmvpj,"Read the T.O.S., you'll thank me later",r/technology
52412,16669,j5m6aj0,t1_j5m0v6m,What am I missing here... https://openai.com/t...,r/technology
52413,16670,j5nylax,t1_j5m0v6m,What does ChatGTP think of its own TOS?,r/technology


In [30]:
reddit_df["comment_body"] = reddit_df["comment_body"].str.lower()

/var/folders/52/p78_6rrn58d3q_81k3cgd9gw0000gn/T/ipykernel_25041/3301605234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df["comment_body"] = reddit_df["comment_body"].str.lower()


In [31]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"i've been shocked for days now, i don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT
3,3,iztfhtb,t3_zj2aeu,"worked on me, ngl.",r/ChatGPT
4,4,izu2as9,t3_zj2aeu,"certified 10/10, must-see moment. it really di...",r/ChatGPT
...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"scary thing is, in this future text won’t be t...",r/technology
52411,16668,j5m0v6m,t3_10jmvpj,"read the t.o.s., you'll thank me later",r/technology
52412,16669,j5m6aj0,t1_j5m0v6m,what am i missing here... https://openai.com/t...,r/technology
52413,16670,j5nylax,t1_j5m0v6m,what does chatgtp think of its own tos?,r/technology


In [32]:
academic_words_list_pattern = '|'.join(academic_words_list)

In [33]:
academic_words_list_pattern

'class|lecture|professor|prof|homework|hw|exam|examinations|assignment|study|degree|gpa|cap|scholarship|scholar|research|thesis|lab|campus|graduation|grad|syllabus|textbook|textbooks|student|students|academic|acad|acads|registrar|tuition|coursework|course|attendance|faculty|teacher|teach|learn|internship|intern|library|peer|peers|school|schools|university|uni|college|colleges'

In [34]:
reddit_df["text_contain_academic"] = reddit_df["comment_body"].str.contains(academic_words_list_pattern)

/var/folders/52/p78_6rrn58d3q_81k3cgd9gw0000gn/T/ipykernel_25041/1998606852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df["text_contain_academic"] = reddit_df["comment_body"].str.contains(academic_words_list_pattern)


In [35]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,text_contain_academic
0,0,iztdxuh,t3_zj2aeu,"i've been shocked for days now, i don't need c...",r/ChatGPT,False
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,True
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT,False
3,3,iztfhtb,t3_zj2aeu,"worked on me, ngl.",r/ChatGPT,False
4,4,izu2as9,t3_zj2aeu,"certified 10/10, must-see moment. it really di...",r/ChatGPT,False
...,...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"scary thing is, in this future text won’t be t...",r/technology,False
52411,16668,j5m0v6m,t3_10jmvpj,"read the t.o.s., you'll thank me later",r/technology,False
52412,16669,j5m6aj0,t1_j5m0v6m,what am i missing here... https://openai.com/t...,r/technology,False
52413,16670,j5nylax,t1_j5m0v6m,what does chatgtp think of its own tos?,r/technology,False


In [36]:
reddit_df["text_contain_academic"].value_counts()

False    38099
True     12943
Name: text_contain_academic, dtype: int64

In [37]:
reddit_acad_df = reddit_df[reddit_df["text_contain_academic"] == True]

In [38]:
reddit_acad_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,text_contain_academic
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,True
11,11,izwlu83,t3_zj2aeu,"i asked to do the opposite of that:\n\n""a titl...",r/ChatGPT,True
23,23,j0bxgq0,t3_zj2aeu,i was messing around and got it to create this...,r/ChatGPT,True
50,50,j0c5wsj,t3_zj2aeu,dead internet theory just with more steps,r/ChatGPT,True
68,68,j18jqxa,t3_zj2aeu,a bunch of stuff i’ve found. \n\ncgpt can draw...,r/ChatGPT,True
...,...,...,...,...,...,...
52398,16655,j4jd7lt,t1_j4j72zf,just wanted to point out that if you lost your...,r/technology,True
52400,16657,j4j9fzg,t1_j4j6qvu,"yeah deepfakes, ai-assisted propaganda and god...",r/technology,True
52402,16659,j4jed9j,t1_j4jbhm0,"well, at least for the moment my job is not in...",r/technology,True
52408,16665,j4kn8zu,t1_j4kk704,"that's not what i said at all.\n\nin fact, i'm...",r/technology,True


In [39]:
reddit_acad_df = reddit_acad_df.drop(columns=['Unnamed: 0',	"comment_id",	"comment_parent_id", "subreddit",	"text_contain_academic"])

In [40]:
reddit_acad_df.to_csv("../cleaned_reddit.csv", index=False)